In [65]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

In [66]:
long_hair_folder = 'long_hair'
short_hair_folder = 'short_hair'

In [67]:
img_height, img_width = 128, 128

In [68]:
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        if img_path.endswith(('.jpg', '.png', '.jpeg')):  # Filter valid image files
            img = load_img(img_path, target_size=(img_height, img_width))
            img_array = img_to_array(img) / 255.0  # Normalize image
            images.append(img_array)
            labels.append(label)
    return np.array(images), np.array(labels)

In [69]:
long_hair_images, long_hair_labels = load_images_from_folder(long_hair_folder, 1)
short_hair_images, short_hair_labels = load_images_from_folder(short_hair_folder, 0)

In [70]:
X = np.concatenate((long_hair_images, short_hair_images), axis=0)
y = np.concatenate((long_hair_labels, short_hair_labels), axis=0)

In [71]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))  # Dropout for regularization

model.add(layers.Dense(1, activation='sigmoid'))

In [73]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(X_train, y_train,epochs=20,batch_size=32,validation_data=(X_val, y_val))

In [76]:
model.save('hair_length_classifier.keras')

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix, precision_score, recall_score
y_pred = model.predict(X_val)
y_pred = np.round(y_pred).astype(int) 

In [ ]:
conf_matrix = confusion_matrix(y_val, y_pred)

plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
precision = precision_score(y_val, y_pred)
print("Precision Score:", precision)

In [ ]:
recall = recall_score(y_val, y_pred)
print("Recall Score:", recall)